<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/03_workshop_parameter_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np

In [2]:
def create_model():
  model = Sequential()
  model.add(Dense(12, input_dim = 8, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # Compile
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [3]:
url = 'https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_keras_sklearn_introduction/diabetes.csv'
data = pd.read_csv(url)
data_values = data.values

In [4]:
X = data_values[:,0:8]
y = data_values[:,8]

In [5]:
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size = 10, verbose = 0)

In [6]:
# StratifiedKFold conserva la distribución de clases del conjunto de datos en las divisiones
# Recuerda el concepto de startificado
kfold = StratifiedKFold(n_splits=10, shuffle=True)

In [7]:
result = cross_val_score(model, X, y, cv = kfold)

In [8]:
print(result.mean())

0.7161654174327851


# Optimizacion de hiperparametros

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
def create_model_optimizer(optimizer = 'rmsprop', init = 'glorot_uniform'):
  model = Sequential()
  model.add(Dense(12, input_dim = 8, kernel_initializer=init, activation='relu'))
  model.add(Dense(8, kernel_initializer=init, activation='relu'))
  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
  return model

In [11]:
model = KerasClassifier(build_fn=create_model_optimizer, verbose = 0)

optimizer = ['rmsprop','adam']

inits = ['glorot_uniform','normal','uniform']
epochs = [50,100,150]
batches = [5,10,20]

In [12]:
param_grid = dict(optimizer=optimizer, epochs = epochs,  batch_size = batches, init = inits)

In [13]:
grid = GridSearchCV(estimator=model,param_grid = param_grid, cv = 3)

In [15]:
grid_result = grid.fit(X,y)

In [16]:
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) con %r" % (mean, stdev, param) )

Mejor: 0.748698 usando {'batch_size': 5, 'epochs': 150, 'init': 'normal', 'optimizer': 'adam'}
0.686198 (0.027126) con {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.710938 (0.011500) con {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.709635 (0.012075) con {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
0.714844 (0.014616) con {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'adam'}
0.703125 (0.000000) con {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.720052 (0.020752) con {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'adam'}
0.704427 (0.018688) con {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.699219 (0.022097) con {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.733073 (0.025976) con {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer': 'rmsprop'